In [4]:
!unzip -o bank-additional.zip

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


In [5]:
import pandas as pd
import sagemaker
import boto3

In [6]:
data = pd.read_csv("./bank-additional/bank-additional-full.csv")
print(data.shape)

(41188, 21)


In [7]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [23]:
import sagemaker
prefix = 'sagemaker/DEMO-smprocessing/input'
input_data = sagemaker.Session().upload_data(
    path="./bank-additional/bank-additional-full.csv",
    key_prefix=prefix)

In [8]:
region = boto3.Session().region_name
print(region)

eu-west-1


In [10]:
role = sagemaker.get_execution_role()    

Couldn't call 'get_role' to get Role ARN from role name Huzaifa_AI_Expert to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::687822941598:user/Huzaifa_AI_Expert, therefore it cannot be used as a SageMaker execution role

In [24]:
def get_execution_role():
    region = boto3.Session().region_name
    client = boto3.client("iam", region_name=region)
    response_roles = client.list_roles(
        PathPrefix="/",
        MaxItems=999
    )
    for role in response_roles["Roles"]:
        if role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
            return role["Arn"]
        raise Exception("Could not resolve what should be the sagemaker role to be used")
        
    role = get_execution_role()
    return role

In [25]:
role = get_execution_role()

In [26]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                    role=role,
                                    instance_type="ml.m5.xlarge",
                                    instance_count=1)

In [28]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(code="preprocessing_original.py",
                     inputs=[ProcessingInput(
                     source=input_data,
                     destination='opt/ml/processing/input')],
                     outputs=[ProcessingOutput(output_name='train_data',
                                              source='opt/ml/processing/train'),
                             ProcessingOutput(output_name="test_data",
                                             source="opt/ml/processing/test")],
                     arguments=["--train-test-split-ration", '0.2'])


Job Name:  sagemaker-scikit-learn-2022-06-07-05-36-23-255
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-687822941598/sagemaker/DEMO-smprocessing/input/bank-additional-full.csv', 'LocalPath': 'opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-687822941598/sagemaker-scikit-learn-2022-06-07-05-36-23-255/input/code/preprocessing_original.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-1-687822941598/sagemaker-scikit-learn-2022-06-07-05-36-23-255/output/train_data', 'LocalPath': 'opt/ml/processing/train', 'S3UploadM

ClientError: An error occurred (ValidationException) when calling the CreateProcessingJob operation: Invalid LocalPath "opt/ml/processing/input" for ProcessingInput "input-1". Please supply an absolute path for LocalPath that begins with "/opt/ml/processing", such as "/opt/ml/processing/input" or "/opt/ml/processing/output".

In [ ]:
Output exceeds the size limit. Open the full output data in a text editor

Job Name:  sagemaker-scikit-learn-2021-09-25-16-27-01-743
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': 
           {'S3Uri': 's3://sagemaker-us-east-1-203856535362/
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': 
            {'S3Uri': 's3://sagemaker-us-east-1-203856535362/sagemaker-scikit-learn-2021-09-25-16-27-01-743/output/train_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'test_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-203856535362/sagemaker-scikit-learn-2021-09-25-16-27-01-743/output/test_data', 'LocalPath': '/opt/ml/processing/test', 'S3UploadMode': 'EndOfJob'}}]